# Preparations
First we are going to load a few packages to perform calculations and visualize the results:

In [ ]:
using Plots, LaTeXStrings, Interact, LinearAlgebra

For interactive plots (uncomment if interactve plots don't work, should only be needed once):

In [ ]:
#using WebIO
#WebIO.install_jupyter_nbextension() # uncomment on first run

Next we define the $2\times 2$ unit matrix $\sigma_0$ and the well known Pauli matricies $\sigma_{x,y,z}$, since we are going to use them a few times. 

In [ ]:
sigma0=Matrix{ComplexF64}(I, 2, 2);
sigmax=Matrix{ComplexF64}([0.0 1.0; 1.0 0.0]);
sigmay=Matrix{ComplexF64}([0.0 -im; im 0.0]);
sigmaz=Matrix{ComplexF64}([1.0 0.0; 0.0 -1.0]);

# Introduction: Physics and aim
In my research project we investigate so called two-dimensional topological insulators (2DTIs) and the helical edge states in these systems.
These are effective 1D states at the edge of a 2D system with certain properties.
For more information see [Qi/Zhang: Topological insulators and superconductors](https://arxiv.org/abs/1008.2026v1), Chapter II.
More specifically we study the density of states and transmission if impurities with finite electric potential and magnetic moment are placed on the edge.
The Hamiltonian for these systems reads
$$
H(t) = \hbar v_F k  \sigma_z + \sum_j (V\sigma_0+\mathbf{M}(t) \cdot \mathbf{\sigma})\delta(x-x_j)
$$
Note, that we have included a time dependence in the magnetic moment.
We have previously investigated the [static case](https://doi.org/10.1103/PhysRevB.98.165423) and a natural extension of the model use there harmonically rotating magnetic impurities.
We have build on the Green's function (GF) formalism for the static case by performing a Fourier decomposition resulting in a Matrix equation that can be solved numerically.
Here we want to focus on one of the paramerters of these numerical calcuations, namely the number of Fourier components needed for reasonable results.

# Floquet Hamiltonian and homogenous fields
It turns out that the GF approach we are using is closely related to the Floquet formalism (see e.g. [Griffoni/Hänggi: Driven quantum tunneling](https://www.physik.uni-augsburg.de/theo1/hanggi/Papers/213.pdf), Chapter II) frequently used to perform calculations on periodically driven systems.
The case of homogenous fields can be viewed as a continous limit of the impurity case and we can explicityly calculate the a matrix representation of the Floquet Hamiltonian for the homogenous case. 
Formally, it can be written as
$$
\mathcal{H}_{m,n;j,k} = <j|H^{m-n}|k> + n \hbar \Omega \delta_{n,m} \delta_{j,k}
$$
where
$$
H^{m-n} = \frac{1}{T} \int_0^T dt \, H(t) \exp{(-i(m-n)\Omega t)}.
$$
For simplicity we use the Hamiltonian
$$
H(t) = \hbar\nu_F k \sigma_z + M (\sin{(\Omega t)}\sigma_z + \cos{(\Omega t)}\sigma_x).
$$
Neglecting the potential $V$ is motivated by the fact, that a constant electric field will only shift the band structure and not have any other impact, and we simply assume we rotate the magnetic field around the edge of the material (that is the y-axis) staying perpendicular to it at all times.
The resulting matix reads
$$
\left(\begin{array}{c|c|c|c|c}
\ddots &\ddots & & & \\ \hline
\ddots & \hbar \nu_F k \sigma_z -\hbar\Omega\sigma_0 & -\frac{i}{2}M (\sigma_x + i \sigma_z) & & \\ \hline
& \frac{i}{2}M (\sigma_x - i \sigma_z) & \hbar \nu_F k \sigma_z & -\frac{i}{2}M (\sigma_x + i \sigma_z) & \\ \hline
& & \frac{i}{2}M (\sigma_x - i \sigma_z) & \hbar \nu_F k \sigma_z +\hbar\Omega\sigma_0 &\ddots \\ \hline
& & &\ddots &\ddots
\end{array}\right).
$$
This matrix now has block-tri-diagonal structure with $2\times 2$ blocks and we can make use of the Pauli matrices we defined earlier to write a function that returns this matrix truncated after a certain number of blocks $n_{max}$:

In [ ]:
"""
    H(k,omega,nmax,M)

Compute the matrix representation of the Floquet Hamiltonian at momentum 'k' with driving frequency 'omega' cutting of after 'nmax' Floquet components using magnetic field strength 'M'
"""
function H(k,omega,nmax,M)
    H=Matrix{ComplexF64}(I, 2*(2*nmax+1), 2*(2*nmax+1)) # initialize a matrix of the correct size
    n=-nmax:nmax # prefactors for the diagonal blocks
    for i=1:2:2*(2*nmax+1)-2
        # fill diagonal
        H[i:i+1,i:i+1]=n[Int((i+1)/2)]*omega*sigma0-k*sigmaz
        # fill off diagonals
        H[i:i+1,i+2:i+3]=-im/2*M*(sigmax+im*sigmaz) # upper
        H[i+2:i+3,i:i+1]=im/2*M*(sigmax-im*sigmaz) # lower
    end
    H[end-1:end,end-1:end]=n[end]*omega*sigma0-k*sigmaz # last block
    return H
end  

The number of blocks $n_{max}$ corresponds to the number of Fourier components of the GF in the approach we are using for the impurity system.
The energy eigenvalues $E$ of this matrix depending on the momentum $k$ gives the Floquet band structure of the system.
Floquet theory also gives a criterion when this Hamiltonian can be used, namely there is a periodicity condition for the eigenvalues.
Since in this case we have $2 \times  2$ blocks in this case we will find two sets of eigenvalues, where the sets each follow the periodicity condition $\varepsilon = \varepsilon_0 + n\hbar\Omega$.
This also makes clear why it is important to choose a suitable number of Fourier/Floquet components: too many will slow down the calculations while not enough will give wrong results.

The block-tri-diagonal structure gives a intitive picture of what we expext to see.
If the magnetic field is turned of we simply get copies of the linear dispersion known for the helical edge states shifted by $n\hbar\Omega$, so-called Floque-sub-bands.
If we turn on the magnetic field the off-diagonal blocks are non-zero and couple the Floquet-sub-bands, such that the crossings turn into avoided crossings.
This can be seen in the interactive graphic below.
We can also clearly see that the periodicity is fulfilled in the fast driving case already for very few Floquet bands while we need significantly more for small driving frequencies.

In [ ]:
@manipulate throttle=0.1 for Ω=0.2:0.2:25, n=1:25, M=0:0.1:1 # throttle for better reactivity, sliders for Ω,n_max,M
    kmax = maximum([Ω,3]) # set maximum momentum k, at least 3 for good visibility for small Ω
    k=range(-kmax,kmax,length=300) # set range for momentum k
    E=eigvals(H(k[1],Ω,n,M)) # calculate Energy eigenvalues for first k
    for i in k[2:end] # loop over remaining k
        E=hcat(E,eigvals(H(i,Ω,n,M))) # append to already calculated eigenvalues
    end
    # and plot
    plot(k,E',legend=false,lc=:red,xlabel=L"\hbar\nu_F k",ylabel=L"E",title="Floquet band structure for homogenous fields")
end

# Comparison to Green's function results

We can also explicitly calculate the zeroth Fourier component of the GF for small numbers of Fourier components and use the spectral function to visualize the bandstructure.
For two Fourier components it can be written as
$$
G_0(E,k)=\left( 1-\tilde{g}(E,k)\left( V_+ \frac{\tilde{g}(E+\hbar \Omega,k)}{1-\tilde{g}(E+\hbar \Omega,k)V_+\tilde{g}(E+2\hbar \Omega,k)V_+^\dagger} V_+^\dagger + V_+^\dagger \frac{\tilde{g}(E-\hbar \Omega,k)}{1-\tilde{g}(E-\hbar \Omega,k)V_+^\dagger\tilde{g}(E-2\hbar \Omega,k)V_+} V_+\right)\right)^{-1}\tilde{g}(E,k),
$$
with the Fourier components of the driving
$$
V_\pm = \frac{M}{2}(\sigma_x \mp i \sigma_z)
$$
and the free GF
$$
\tilde{g}(E,k) = \frac{1}{(E+i0^+)-k\sigma_z}.
$$
Note that we have added a small imaginary part for convergence (needed for integrations over $k$ via the residual theorem).
The spectral function is then defined as 
$$
A(E,k) = -\mathrm{Im} \, \mathrm{Tr} \, G_0(E,k)
$$
and we can write a function for it.

First define a function for the free GF and the Fourier components of the driving (with $M=1$ for simplicity):

In [ ]:
g0(E,k)=inv(sigma0*(E+im*10e-3)-k*sigmaz)
Vp=1/2*(sigmax-im*sigmaz)
Vm=1/2*(sigmax+im*sigmaz);

Using these we can define the spectral function:

In [ ]:
A(E,k,omega)=-imag(tr(inv(sigma0-g0(E,k)*(Vp*inv(sigma0-g0(E+omega,k)*Vp*g0(E+2*omega,k)*Vm)*g0(E+omega,k)*Vm+Vm*inv(sigma0-g0(E-omega,k)*Vm*g0(E-2*omega,k)*Vp)*g0(E-omega,k)*Vp))*g0(E,k)))

We can calculate this on a suitable range of energies and momenta using the map function (for now $\Omega = 25$, we can use the interactive plot above to find a good range for $E,k$)

In [ ]:
Emax = 75
kmax = 25
Erange = range(-Emax,Emax,length=300)
krange = range(-kmax,kmax,length=300)
Ekrange = [(E,k) for E in Erange, k in krange]
spectral = map(x->A(x...,25),Ekrange);

and plot the results as a heatmap:

In [ ]:
plot(krange,Erange,spectral,linetype=:heatmap,clims=(0,10e-4*maximum(spectral)),colorbar_title="Spectral function A",xlabel=L"\hbar\nu_F k",ylabel=L"E",size=(900,400),margins = 5mm)

Here we can see that the subbands carry different weight and that for this particular fast driving case with already the second set of sub bands i almost invisible.
This hints at the cut off after two Fourier coefficients being good as expected from the Floquet band structure above.
We can also put the Floquet band structure on top to compare and take a closer look at the avoided crossings.
For this first calculate the Floquet band structure as before and the spectral function with higher resolution around one of the crossing:

In [ ]:
# Floquet band structure
E=eigvals(H(krange[1],25,2,1)) # calculate Energy eigenvalues for first k
for i in krange[2:end] # loop over remaining k
   E=hcat(E,eigvals(H(i,25,2,1))) # append to already calculated eigenvalues
end
# higher resolution for inset
Erange_ = range(10,15,length=300)
krange_ = range(10,15,length=300)
Ekrange_ = [(E,k) for E in Erange_, k in krange_]
spectral_ = map(x->A(x...,25),Ekrange_);

Now we can plot the hearmap, overlay it with the Floquet bandstructure and so the same for the inset:

In [ ]:
hm=plot(krange,Erange,spectral,linetype=:heatmap,clims=(0,10e-4*maximum(spectral)),colorbar_title="Spectral function A",xlabel=L"\hbar\nu_F k",ylabel=L"E",size=(900,400), margins = 5mm)
plot!(hm,krange,E',lc=:red,legend=false,widen=false)
plot!(krange_,Erange_,spectral_,linetype=:heatmap,xlims=(10,15),ylims=(10,15),clims=(0,10e-4*maximum(spectral)),cbar=false,inset_subplots = [(1, bbox(0.0,0.02,0.5,0.3,:center,:top))], subplot=2,fg_text=:white)
plot!(hm[2],krange,E',lc=:red,legend=false,widen=false,xlims=(10,15),ylims=(10,15))

In [ ]:
# save to file
# savefig("hf_inset.pdf") # pdf, gets rather big with heatmap
savefig("hf_inset.png")

So far we looked at the fast driving case, where we only need a few Fourier/Floquet components.
Now lets look at a faster driving case and create similar figures.
For now we choose $\Omega = 2$ and overlay the spectral function from the GF as before with the Floquet band structure for $n_{max} = 2$ and $n_{max} = 10$.

In [ ]:
# spectral function
Emax = 5
kmax = 9
Erange = range(-Emax,Emax,length=300)
krange = range(-kmax,kmax,length=300)
Ekrange = [(E,k) for E in Erange, k in krange]
spectral2 = map(x->A(x...,2),Ekrange);
# Floquet band structure
E2=eigvals(H(krange[1],2,2,1)) # calculate Energy eigenvalues for first k
for i in krange[2:end] # loop over remaining k
   E2=hcat(E2,eigvals(H(i,2,2,1))) # append to already calculated eigenvalues
end
E10=eigvals(H(krange[1],2,10,1)) # calculate Energy eigenvalues for first k
for i in krange[2:end] # loop over remaining k
   E10=hcat(E10,eigvals(H(i,2,10,1))) # append to already calculated eigenvalues
end

In [ ]:
hm=plot(krange,Erange,spectral2,linetype=:heatmap,clims=(0,10e-3*maximum(spectral)),colorbar_title="Spectral function A",xlabel=L"\hbar\nu_F k",ylabel=L"E",ylims=(-Emax,Emax),size=(900,400))
plot!(hm,krange,E2',lc=:red,legend=false,widen=false)
plot!(hm,krange,E10',lc=:grey,ls=:dash,legend=false,widen=false)
#plot!(krange_,Erange_,spectral_,linetype=:heatmap,xlims=(10,15),ylims=(10,15),clims=(0,10e-4*maximum(spectral)),cbar=false,inset_subplots = [(1, bbox(0.0,0.02,0.5,0.3,:center,:top))], subplot=2,fg_text=:white)
#plot!(hm[2],krange,E',lc=:red,legend=false,widen=false,xlims=(10,15),ylims=(10,15))

In [ ]:
#save to file
#savefig("med_noinset.pdf") # pdf, gets rather big with heatmap
savefig("med_noinset.png")

Here we see that the  periodicity is not fulfilled for the red case but the results in the end are not too different because the sub bands carrying significant weight still match rather well.
If we calculate the density of states from the spectral function here (integrate over $k$) er eill only see a slight shift in the position of the outermost peaks.

Now finally we want to look at a case where we certainly need more Fourier/Floquet components.
For this we will disregard the rather slow spectral function calculations and only compare the Floquet band structures.
Choosing $\Omega = 0.2$ and $n_{max}=10,25$.

In [ ]:
# set limits and calculate Floquet bandstructure
Emax = 2
kmax = 3
krange = range(-kmax,kmax,length=300)
E1=eigvals(H(krange[1],0.2,10,1)) # calculate Energy eigenvalues for first k
for i in krange[2:end] # loop over remaining k
   E1=hcat(E1,eigvals(H(i,0.2,10,1))) # append to already calculated eigenvalues
end
E25=eigvals(H(krange[1],0.2,25,1)) # calculate Energy eigenvalues for first k
for i in krange[2:end] # loop over remaining k
   E25=hcat(E25,eigvals(H(i,0.2,25,1))) # append to already calculated eigenvalues
end
# plot
pltleft=plot(krange,E1',lc=:red,label=["\$ n_{max}=$(10) \$" reshape(["" for i in 1:size(E1)[1]-1],1,size(E1)[1]-1)],xlims=(-kmax,kmax),ylims=(-Emax,Emax),borderstyle=:box,ylabel=L"E")
pltright=plot(krange,E25',lc=:blue,label=["\$ n_{max}=$(25) \$" reshape(["" for i in 1:size(E25)[1]-1],1,size(E25)[1]-1)],xlims=(-kmax,kmax),ylims=(-Emax,Emax),borderstyle=:box)
pltboth=plot(pltleft,pltright,layout=(1,2),size=(1000,400),xlabel=L"\hbar\nu_F k",guidefontsize=22,legend=:top,legendfontsize=14,xticks=-kmax:kmax,tickfontsize=14, margins = [5mm 10mm])

In [ ]:
#save to file
savefig("lf_noinset.pdf")
savefig("lf_noinset.png")

Here we can clearly see that on the left the periodicity is not respected, whil it is fulfilled on the right.
To guarantee proper results in the final results we can now use the same procedure for the desired driving frequency and check where we cut of the Fourier series. 

Using the type of plots above it is possible to easily estimate if the number of fourier coefficients chosen for the driving frequency is sufficient or not.